In [1]:
import rasterio
import rasterio.warp
import matplotlib.pyplot as plt
import affine
import numpy
import pyproj
import pandas

import starepandas
import pystare

# Load Landsat Granule/Scene

In [2]:
fname = '../tests/data/LC08_L2SP_042036_20210729_20210804_02_T1_SR_B1.TIF'
with rasterio.open(fname) as dataset:
    data = dataset.read()
    mask = dataset.dataset_mask()
    crs = dataset.crs

    # Pixel center instead of pixel corner
    transform = dataset.transform * affine.Affine.translation(0.5, 0.5)

In [3]:
data = data[0, 0:500, 0:500]

In [4]:
indices = numpy.indices(data.shape, dtype='int16')
xy = numpy.array(transform*indices, dtype='int32')

In [5]:
transformer = pyproj.Transformer.from_crs(crs, "epsg:4326")
lat, lon = transformer.transform(xy[0], xy[1])

In [6]:
sids = pystare.from_latlon_2d(lat, lon, adapt_level=True)

In [7]:
landsat = starepandas.STAREDataFrame({'lon': lon.flatten(),
                                      'lat': lat.flatten(),
                                      'band_1': data.flatten()})

In [8]:
landsat = starepandas.STAREDataFrame(landsat, sids=sids.flatten())

# High-Level

In [9]:
# starepandas.read_geotiff(fname, add_xy=True, add_coordinates=True, add_latlon=True, add_sids=False)

,band_1,lon,lat,ix,iy,x,y
0,0,-121.173073,35.644070,0.0,0.0,122100.0,3952500.0
1,0,-121.172745,35.644081,1.0,0.0,122130.0,3952500.0
2,0,-121.172417,35.644093,2.0,0.0,122160.0,3952500.0
3,0,-121.172081,35.644104,3.0,0.0,122190.0,3952500.0
4,0,-121.171753,35.644115,4.0,0.0,122220.0,3952500.0
...,...,...,...,...,...,...,...
62342586,0,-118.541695,33.553364,7826.0,7960.0,356880.0,3713700.0
62342587,0,-118.541374,33.553368,7827.0,7960.0,356910.0,3713700.0
62342588,0,-118.541054,33.553371,7828.0,7960.0,356940.0,3713700.0
62342589,0,-118.540726,33.553375,7829.0,7960.0,356970.0,3713700.0
